Server command:

```bash
# Setup
source ~/miniconda3/bin/activate
conda create --prefix ./envs python=3.10
pip install -e .
llama download --model-id Llama3.2-11B-Vision-Instruct
llama download --model-id Llama-Guard-3-1B

export LLAMA_STACK_PORT=5001
export INFERENCE_MODEL=meta-llama/Llama-3.2-11B-Vision-Instruct
export INFERENCE_PORT=8000
export SAFETY_MODEL=meta-llama/Llama-Guard-3-1B

# Start server
source ~/miniconda3/bin/activate
conda activate ./envs
llama stack build --template meta-reference-gpu --image-type conda && track llama stack run distributions/meta-reference-gpu/run-with-safety.yaml \
  --port 5001 \
  --env INFERENCE_MODEL=meta-llama/Llama-3.2-11B-Vision-Instruct
```

In [1]:
import uuid
from llama_stack_client import LlamaStackClient
from llama_stack_client.types.memory_insert_params import Document

client = LlamaStackClient(
    base_url="http://localhost:5001",
)

providers = client.providers.list()
memory_banks_response = client.memory_banks.list()

bank_id = f"bank_{uuid.uuid4()}"
provider = providers["memory"][0]
client.memory_banks.register(
    memory_bank_id=bank_id,
    params={
        "embedding_model": "all-MiniLM-L6-v2",
        # Is the default for agent config: https://github.com/meta-llama/llama-stack/blob/66d8f4ffd126bff668434b314892a99fe854a034/llama_stack/providers/inline/agents/meta_reference/agent_instance.py#L668
        "chunk_size_in_tokens": 512,
    },
    provider_id=provider.provider_id,
)
bank_id

'bank_496762bf-7a55-435f-9133-7520f8769673'

In [5]:
with open("long_story.txt", "r") as f:
    story = f.read()

document = Document(
    document_id=str(uuid.uuid4()),
    content=story,
    metadata={"source": "long_story.txt"}
)

client.memory.insert(
    bank_id=bank_id,
    documents=[document],
)